In [1]:
cd /kaggle/input/bart-utilities

/kaggle/input/bart-utilities


In [2]:
import sys
from BART_utilities import *
sys.path.insert(0, '../')


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
cd /kaggle/input/utilis

/kaggle/input/utilis


In [4]:
from utilities import *
import transformers
import pandas as pd
import numpy as np
import glob
import math
import random
import re
import argparse
import nltk
from transformers import Trainer, TrainingArguments

In [5]:
import os
dataset = "IN-Abs"

#Reading the test documents
names, data_source, data_summary = get_summary_data(dataset, "train")

In [6]:
#cleaning the data
cleaned_data_source=list()
cleaned_data_summary=list()

for data in data_source:
    cleaned_text = data.replace('\n', ' ').replace('\t', ' ')
    cleaned_data_source.append(cleaned_text)

for data in data_summary:
    cleaned_text = data.replace('\n', ' ').replace('\t', ' ')
    cleaned_data_summary.append(cleaned_text)
import pandas as pd

df = pd.DataFrame(list(zip(cleaned_data_source[:5],cleaned_data_summary[:5])),columns =['data', 'summary'])

# Display the DataFrame
df.head()

,data,summary
0,iminal Appeal No. 47 of 1963. Appeal by specia...,"The appellant published in his paper, which ha..."
1,minal Appeal No. 210 of 1963. Appeal by specia...,On the first information report lodged by the ...
2,Civil Appeal No. 2618 of 1983. From the Judgme...,The first respondent who lost to the appellant...
3,ivil Appeal No. 2991 of 1986. From the Judgmen...,The appellant was sought to be evicted under s...
4,Civil Appeal No. 1733 of 1973. From the Judgme...,A firm (JTC) obtained the sole selling agency ...


In [7]:
df.rename(columns = {'data':'source', 'summary':'target'}, inplace = True)

In [8]:
# Loading Model and tokenizer
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW, BartConfig

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large', add_prefix_space=True)


bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")
bart_model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerN

In [9]:
# Add special tokens if required

new_tokens = ['<F>', '<RLC>', '<A>', '<S>', '<P>', '<R>', '<RPC>']

special_tokens_dict = {'additional_special_tokens': new_tokens}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
bart_model.resize_token_embeddings(len(tokenizer))

Embedding(50272, 1024)

In [10]:
import torch

In [11]:

summary_data = SummaryDataModule(tokenizer, df, batch_size = 1)
model = LitModel(learning_rate = 2e-5, tokenizer = tokenizer, model = bart_model)
model.to(device)

LitModel(
  (model): BartForConditionalGeneration(
    (model): BartModel(
      (shared): Embedding(50272, 1024)
      (encoder): BartEncoder(
        (embed_tokens): Embedding(50272, 1024)
        (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
        (layers): ModuleList(
          (0-11): 12 x BartEncoderLayer(
            (self_attn): BartAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1024, bias=True)
           

In [12]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

from pytorch_lightning.callbacks import LearningRateFinder

# Create TensorBoardLogger with a writable directory
logger = TensorBoardLogger(save_dir="/kaggle/working/", name="my_logs")
torch.set_grad_enabled(True)
trainer = pl.Trainer(max_epochs= 1,accelerator='gpu',min_epochs = 2,precision = 16,logger=logger)

/opt/conda/lib/python3.10/site-packages/lightning_fabric/connector.py:565: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!


In [13]:
trainer.fit(model, summary_data)

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:293: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [14]:
trainer.save_checkpoint("/kaggle/working/output_model.ckpt")

In [15]:
torch.save(model.state_dict(),"/kaggle/working/outputBARTModel.pt")

In [16]:
# new_model = LitModel(learning_rate = 2e-5, tokenizer = tokenizer, model = model)
# new_model=LitModel.load_from_checkpoint("/kaggle/working/output_model.ckpt")
# model = LitModel(learning_rate = 2e-5, tokenizer = tokenizer, model = bart_model)
# model.to(device)

# finetuned_model = LitModel(learning_rate = 2e-5, tokenizer = tokenizer, model = bart_model)

In [17]:
# finetuned_model.load_state_dict(torch.load("/kaggle/working/outputBARTModel.pt"))

In [18]:
# summaries=list()
# nested="hello how are you"
# input_tokenized = tokenizer.encode(nested, truncation=True, return_tensors='pt')
# input_tokenized = input_tokenized.to(device)
# summary_ids = finetuned_model.model.to(device).generate(input_tokenized,
#                                   length_penalty=0.01,
#                                   min_length=50,
#                                   max_length=100)
# output = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
# summaries.append(output)

# print(summaries)

In [19]:

# from transformers import BartForConditionalGeneration

# # Replace '/kaggle/working/output_model' with the actual path to your saved model directory
# model_path = '/kaggle/input/bart-finetunedmodel-input/kaggle/working/output_model'

# finetuned_model = BartForConditionalGeneration.from_pretrained(model_path)

In [20]:
# from transformers import BartForConditionalGeneration

# finetuned_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')  # Replace with your model's name

# checkpoint_path = '/kaggle/working/output.ckpt'  # Replace with your checkpoint file path
# state_dict = torch.load(checkpoint_path, map_location='cpu')  # Use 'cuda' if loading on GPU

# # Remove unwanted keys
# unwanted_keys = ['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers', 'MixedPrecision', 'hyper_parameters']
# state_dict = {k: v for k, v in state_dict.items() if k not in unwanted_keys}

# # Load the state dictionary into the model
# model.load_state_dict(state_dict, strict=False)


In [21]:
# from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# # Load tokenizer and model
# model_name = 'model'  # Replace with your fine-tuned model name
# tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large', add_prefix_space=True)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# # fine_tuned_model = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/working/output")

# # Example text to summarize
# input_text = "On March 5, 2021, the Securities and Exchange Commission"

# # Tokenize and encode input text
# inputs = tokenizer(input_text, return_tensors='pt')

# # Generate summaries
# with torch.no_grad():
#     outputs = model.generate(
#         input_ids=inputs['input_ids'],
#         attention_mask=inputs['attention_mask'],
#         max_length=150,  # Adjust as needed
#         num_beams=4,     # Adjust as needed
#         length_penalty=2.0,  # Adjust as needed
#         early_stopping=True
#     )

# # Decode the generated summaries
# generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
# print("Generated Summary:", generated_summary)